In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive/MyDrive/nlp"

Mounted at /content/drive
/content/drive/MyDrive/nlp


In [2]:
!pip install -q -U trl transformers accelerate peft datasets bitsandbytes evaluate git+https://github.com/huggingface/huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!nvidia-smi # check if GPU is avaiable

Thu May  2 08:54:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import torch
torch.cuda.is_available() # check if GPU is avaiable

True

In [6]:
from transformers import pipeline, AutoModelForCausalLM, AutoModelForSequenceClassification, BloomForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

model_name = "distilbert/distilbert-base-uncased"
#model_name = "bigscience/bloomz-560m"

In [7]:
from datasets import load_dataset

# Task: BoolQ
data_boolq_train = load_dataset("csv", data_files="slovene_superglue/SuperGLUE-HumanT/csv/BoolQ/train.csv")
data_boolq_val = load_dataset("csv", data_files="slovene_superglue/SuperGLUE-HumanT/csv/BoolQ/val.csv")
data_boolq_test = load_dataset("csv", data_files="slovene_superglue/SuperGLUE-HumanT/csv/BoolQ/test.csv")


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
def preprocess_data(examples):

  passage_inputs = [f"passage : {x} " for x in examples["passage"]]
  question_inputs = [f"question : {x} " for x in examples["question"]]
  inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

  model_inputs = tokenizer(inputs, padding=True, max_length=100, truncation=True)
  model_inputs["labels"] = examples["label"]
  model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

  return model_inputs

tokenized_boolq_train = data_boolq_train.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)
tokenized_boolq_val = data_boolq_val.map(preprocess_data,remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

train = tokenized_boolq_val["train"]

#print(train)
#for i in train['input_ids']:
    #print(tokenizer.decode(i))


Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
from transformers import BertForSequenceClassification

id2label = {0: False, 1: True}
label2id = {False: 0, True: 1}

# RobertaForSequenceClassification
# BertForSequenceClassification
# BloomForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

NUM_VIRTUAL_TOKENS = 4

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    num_layers=6,
    token_dim=768,
    num_attention_heads=12,
    tokenizer_name_or_path=model_name #The pre-trained model
)

In [15]:
peft_model = get_peft_model(model, peft_config)
print(peft_model.print_trainable_parameters())

trainable params: 595,202 || all params: 67,550,212 || trainable%: 0.8811252879561651
None


In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./prompt_fine_tuned_boolq",
    learning_rate= 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    use_cpu=False,
    load_best_model_at_end=True,
    num_train_epochs=6,
)

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_boolq_train["train"],
    eval_dataset=tokenized_boolq_val["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.595139,0.777778
2,No log,0.592964,0.777778
3,No log,0.588423,0.777778
4,No log,0.585476,0.777778
5,No log,0.584592,0.777778
6,No log,0.584642,0.777778


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=72, training_loss=0.6499664518568251, metrics={'train_runtime': 5.7538, 'train_samples_per_second': 95.937, 'train_steps_per_second': 12.514, 'total_flos': 14477754873600.0, 'train_loss': 0.6499664518568251, 'epoch': 6.0})

In [25]:
# save model to hub
model_location = "tjasad/prompt_fine_tuned_boolq"
trainer.push_to_hub(model_location)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

events.out.tfevents.1714640822.4b1c6a2be26d.1397.2:   0%|          | 0.00/7.01k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1714640277.4b1c6a2be26d.1397.1:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

events.out.tfevents.1714640183.4b1c6a2be26d.1397.0:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tjasad/prompt_fine_tuned_boolq/commit/b955f028c388351a7f658664f6e23a42c022048e', commit_message='tjasad/prompt_fine_tuned_boolq', commit_description='', oid='b955f028c388351a7f658664f6e23a42c022048e', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model="tjasad/prompt_fine_tuned_boolq")
classifier(text)

config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.525802731513977}]